In [1]:
import os
import json
import time
import pandas as pd
from src.gender import gender
from src.vycisti import vycisti
pd.set_option('display.max_columns', None)
df = pd.read_json(os.path.join("data", "filmy.json"))
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())
with open(os.path.join("data_fixes", "zeny.json"), encoding="utf-8") as zeny:
    zeny = json.loads(zeny.read())
with open(os.path.join("data_fixes", "nelide.json"), encoding="utf-8") as nelide:
    nelide = json.loads(nelide.read())
df = vycisti(df, opravy)

2023/05/28 22:56:02 z 4555 řádků odstraněno 125. Aktuální dataframe má 4430 řádků.


## Dlouhý výpočet, kterým se nechceme zdržovat v sešitu věnovaném tvůrcům

In [2]:
profese = seznamy["profese"]
persony = pd.read_json(os.path.join("data", "persony.json"))
jmena_lidi = persony[persony["Osoba"] == "fyzická"]["Jméno"].to_list()

In [3]:
profese.remove("Účinkují")
profese.remove("Hudba, text a zpěv písně")
profese.remove("Hudba k písni")

In [4]:
def plose_jednoduse(x):
    seznam = []
    for y in x:
        if y == None:
            pass
        if isinstance(y, str):
            if y not in seznam:
                seznam.append(y)
        if isinstance(y, list):
            for z in y:
                if z not in seznam:
                    seznam.append(z)
    return(seznam)

In [5]:
def kolik_skoncilo(konce, jmena_lidi):
    st = time.time()
    konce["Lidé"] = konce[profese].values.tolist()
    konce["Lidé"] = konce["Lidé"].apply(lambda x: plose_jednoduse(x))
    konce = konce[["Copyright","Lidé","Film"]].explode("Lidé")
    konce = konce[konce["Lidé"].isin(jmena_lidi)]
    konce = konce.groupby("Copyright")["Lidé"].apply(lambda x: list(set(x)))
    konce = pd.DataFrame(konce).explode("Lidé")
    konce = konce.reset_index()  
    print(f"{len(konce)} člověkoletopočtů")
    konce["Příští film"] = konce[["Lidé","Copyright"]].apply(lambda x: konce[(konce["Lidé"] == x["Lidé"]) & (konce["Copyright"] > x["Copyright"])]["Copyright"].min(), axis = 1)
    konce = konce.rename(columns={"Lidé": "Jméno"})
    konce["Pauza"] = konce["Příští film"] - konce["Copyright"]
    print(f"čas výpočtu: {time.time() - st} s")
    return(konce)

In [ ]:
konce = kolik_skoncilo(df, jmena_lidi)

95431 člověkoletopočtů


In [ ]:
konce.to_json(os.path.join("data", "pauzy.json"), orient="records")

In [ ]:
konce.sample(5)

## Explorace

In [ ]:
konce.groupby("Copyright")["Pauza"].mean().plot.bar(figsize=(20,5))

In [ ]:
r3 = konce.groupby("Jméno")["Copyright"].nunique()
r3 = r3[r3 >= 5]
r3 = r3.index.to_list()

In [ ]:
k3 = konce[konce["Jméno"].isin(r3)]

In [ ]:
k3 = k3[k3["Copyright"].isin(range(1920,2011))]

In [ ]:
k3 = k3.merge(persony[["Jméno","Rok úmrtí"]], on="Jméno")

In [ ]:
k3["Příští film"] = pd.to_numeric(k3["Příští film"], errors="coerce")

In [ ]:
k3["Rok úmrtí"] = pd.to_numeric(k3["Rok úmrtí"], errors="coerce")

In [ ]:
k3.loc[k3["Příští film"] > k3["Rok úmrtí"],"Příští film"]=None

In [ ]:
k3.groupby("Copyright")["Jméno"].nunique()

In [ ]:
k3.sample(5)

In [ ]:
pd.Series(k3[k3["Pauza"].isnull()].groupby("Copyright")["Jméno"].nunique() / k3.groupby("Copyright")["Jméno"].nunique()).plot.bar(figsize=(20,5))

In [ ]:
k3[k3["Copyright"] == 1992]

In [ ]:
pd.Series(k3[(k3["Pauza"] >= 12)].groupby("Copyright")["Jméno"].nunique() / k3.groupby("Copyright")["Jméno"].nunique()).plot.bar(figsize=(20,10))

In [ ]:
pd.Series(k3[(k3["Pauza"] >= 15) | (k3["Pauza"].isnull())].groupby("Copyright")["Jméno"].nunique() / k3.groupby("Copyright")["Jméno"].nunique()).plot.bar(figsize=(20,10))

In [ ]:
pd.Series(k3[(k3["Pauza"] >= 10)].groupby("Copyright")["Jméno"].nunique() / k3[~k3["Pauza"].isnull()].groupby("Copyright")["Jméno"].nunique()).plot.bar(figsize=(20,10))